In [1]:
%load_ext autoreload
%autoreload 2

import pandas as pd
from factorgraph import FactorNode, VariableNode, FactorGraph
import json
from collections import namedtuple
import os
import numpy as np

# read d4j data

In [8]:
BugData = namedtuple(
    "BugData", ["pid", "vid", "fault_idxs", "fail_tcs", "covmat"]
)

pid, vid = "Math", 100
covmat_path = f"d4j-data/covmat/{pid}-{vid}.pkl"
covmat = pd.read_pickle(covmat_path)
bugdata_path = f"d4j-data/bugdata/{pid}-{vid}.json"
with open(bugdata_path, "r") as f:
    obj = json.load(f)
bug_data = BugData(
    obj["pid"], obj["vid"], obj["fault_idxs"], obj["fail_tcs"], covmat
)
bug_data.covmat.sort_index(inplace=True)
bug_data.covmat

,org.apache.commons.math.stat.descriptive.SummaryStatisticsImplTest.testN0andN1Conditions,org.apache.commons.math.distribution.ChiSquareDistributionTest.testConsistency,org.apache.commons.math.complex.FrenchComplexFormatTest.testZeroImaginary,org.apache.commons.math.complex.ComplexFormatTest.testPositiveInfinity,org.apache.commons.math.analysis.BisectionSolverTest.testSerialization,org.apache.commons.math.ode.GraggBulirschStoerIntegratorTest.testIncreasingTolerance,org.apache.commons.math.stat.descriptive.moment.MeanTest.testMomentSmallSamples,org.apache.commons.math.stat.descriptive.moment.MeanTest.testSerialization,org.apache.commons.math.complex.ComplexUtilsTest.testSinh,org.apache.commons.math.complex.ComplexFormatTest.testSimpleWithDecimals,...,org.apache.commons.math.linear.BigMatrixImplTest.testTrace,org.apache.commons.math.fraction.FractionFormatTest.testParse,org.apache.commons.math.complex.FrenchComplexFormatTest.testStaticFormatComplex,org.apache.commons.math.linear.QRDecompositionImplTest.testDimensions,org.apache.commons.math.MathExceptionTest.testConstructor,org.apache.commons.math.stat.descriptive.SummaryStatisticsTest.testEqualsAndHashCode,org.apache.commons.math.stat.descriptive.summary.SumSqTest.testEvaluation,org.apache.commons.math.util.TransformerMapTest.testRemoveTransformer,org.apache.commons.math.util.MathUtilsTest.testIndicatorDouble,org.apache.commons.math.stat.CertifiedDataTest.testSummaryStatistics
"(org.apache.commons.math.ArgumentOutsideDomainException$<init><DDD>, 39)",0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
"(org.apache.commons.math.ArgumentOutsideDomainException$<init><DDD>, 42)",0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
"(org.apache.commons.math.ConvergenceException$<init><>, 34)",0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
"(org.apache.commons.math.ConvergenceException$<init><>, 35)",0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
"(org.apache.commons.math.ConvergenceException$<init><Ljava/lang/String;>, 91)",0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
"(org.apache.commons.math.util.TransformerMap$transform<Ljava/lang/Object;>, 146)",0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
"(org.apache.commons.math.util.TransformerMap$transform<Ljava/lang/Object;>, 147)",0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
"(org.apache.commons.math.util.TransformerMap$transform<Ljava/lang/Object;>, 148)",0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
"(org.apache.commons.math.util.TransformerMap$transform<Ljava/lang/Object;>, 152)",0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


## Use index and exec name proxy since index is too long

In [11]:
idxs = bug_data.covmat.index.values
for idx in idxs[:5]:
    print(idx)
idxs_proxy = [f"e{i}" for i in range(len(idxs))]
print(idxs_proxy[:5])

('org.apache.commons.math.ArgumentOutsideDomainException$<init><DDD>', '39')
('org.apache.commons.math.ArgumentOutsideDomainException$<init><DDD>', '42')
('org.apache.commons.math.ConvergenceException$<init><>', '34')
('org.apache.commons.math.ConvergenceException$<init><>', '35')
('org.apache.commons.math.ConvergenceException$<init><Ljava/lang/String;>', '91')
['e0', 'e1', 'e2', 'e3', 'e4']


In [12]:
exec_names = bug_data.covmat.columns.values
for exec_name in exec_names[:5]:
    print(exec_name)
exec_names_proxy = [f"tc{i}" for i in range(len(exec_names))]
print(exec_names_proxy[:5])

org.apache.commons.math.stat.descriptive.SummaryStatisticsImplTest.testN0andN1Conditions
org.apache.commons.math.distribution.ChiSquareDistributionTest.testConsistency
org.apache.commons.math.complex.FrenchComplexFormatTest.testZeroImaginary
org.apache.commons.math.complex.ComplexFormatTest.testPositiveInfinity
org.apache.commons.math.analysis.BisectionSolverTest.testSerialization
['tc0', 'tc1', 'tc2', 'tc3', 'tc4']


## generate covered element list

In [14]:
execs = []
for exec_name in exec_names:
    exec_et = [idx_num for idx_num in range(len(idxs)) if bug_data.covmat[exec_name].iloc[idx_num] > 0]
    execs.append(exec_et)


## generate results

In [15]:
results = []
for exec_name in exec_names:
    if exec_name in bug_data.fail_tcs:
        results.append(1)
    else:
        results.append(0)


## save generated data

In [24]:
output_dir = f"d4j-data/fg-gen-data/{pid}-{vid}"
os.makedirs(output_dir, exist_ok=True)
with open(os.path.join(output_dir, "idxs"), "w") as f:
    f.write("\n".join([str(tup) for tup in idxs]))
with open(os.path.join(output_dir, "exec_names"), "w") as f:
    f.write("\n".join(exec_names))
exec_df = pd.DataFrame(execs)
exec_df.to_csv(os.path.join(output_dir, "execs"), index=False)
with open(os.path.join(output_dir, "results"), "w") as f:
    f.write(" ".join([str(result) for result in results]))

## load data

In [2]:
pid, vid = "Math", 100
output_dir = f"d4j-data/fg-gen-data/{pid}-{vid}"
if not os.path.isdir(output_dir):
    print(f"No generated data: ({output_dir})")
else:
    with open(os.path.join(output_dir, "idxs"), "r") as f:
        idxs = [eval(line.strip()) for line in f.readlines()]
    with open(os.path.join(output_dir, "exec_names"), "r") as f:
        exec_names = [line.strip() for line in f.readlines()]
    exec_df = pd.read_csv(os.path.join(output_dir, "execs"))
    execs = exec_df.values
    execs = [[int(linenum) for linenum in exec_et if not np.isnan(linenum)] for exec_et in execs]
    with open(os.path.join(output_dir, "results"), "r") as f:
        results = [int(result) for result in f.read().split()]

idxs_proxy = [f"e{i}" for i in range(len(idxs))]
exec_names_proxy = [f"tc{i}" for i in range(len(exec_names))]


## Create factor graph

In [3]:
variable_list = [VariableNode(idx) for idx in idxs_proxy]
for exec_name, result in zip(exec_names_proxy, results):
    trnode = VariableNode(exec_name)
    trnode.val = result
    trnode.fix()
    variable_list.append(trnode)
len(variable_list)

def prob_succ(elem_faulty):
    # This one is ad-hoc one.
    num_f = sum(elem_faulty)
    num_c = len(elem_faulty) - num_f
    return 0.01 ** (num_f / 1.1 ** num_c) 

def prob_match(result, elem_faulty):
    return 1 - prob_succ(elem_faulty) if result else prob_succ(elem_faulty)

scope = {"prob_succ": prob_succ, "prob_match": prob_match}

factor_list = []
for exec_name, et in zip(exec_names_proxy, execs):
    covered_elems = [idxs_proxy[i] for i in et]
    param_str = ", ".join([f"V[{elem}]" for elem in covered_elems])
    factor_node = FactorNode(
        exec_name, f"prob_match(V[{exec_name}], [{param_str}])")
    factor_list.append(factor_node)


factor_graph = FactorGraph(variable_list, factor_list)

In [4]:
print(factor_graph.size, factor_graph.size_factor)

9650 1101


In [5]:
factor_graph.marginal_sum_product(scope, debug=True)

Initialize meesage dict (time : 21.813314199447632, memory: 232 bytes)
iteration 1 starts
v2f message start...
v2f message done (time : 10.645788192749023)
f2v message start...
0 fac2vars done... (0.0%)


KeyboardInterrupt: 